In [229]:
import pymysql
import pandas as pd

In [230]:
df = pd.read_csv('mysite/data/features.csv')
features = df.loc[:, df.columns != 'id']
coffees = df.reset_index(drop=False)
coffees = coffees.rename(columns={"index":"coffeeid"})

# MySQL 연결 정보
mysql_host = 'localhost'
mysql_user = 'root'
mysql_password = 'MShw1214!'
mysql_db = 'wondoodoo'

conn = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_password, database=mysql_db)

try:
    # 쿼리 작성
    review_query = f"SELECT * FROM review;"
    users_query = f"SELECT * FROM main_preference;"

    # MySQL에서 데이터 읽어오기
    users = pd.read_sql(users_query, conn)  # ['id', 'caf', 'blend', 'notes', 'sour', 'sweet', 'bitter', 'body', 'user_id']
    users.reset_index(drop=False, inplace=True)
    
    ratings = pd.read_sql(review_query, conn)   # 'id', 'Stars', 'content', 'created_date', 'Coffee_id', 'Order_id', 'user_id'
    ratings = ratings[['user_id', 'Coffee_id', 'Stars', 'created_date']]


    print(f"데이터({users.shape})를 성공적으로 불러왔습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # 연결 닫기
    conn.close()


데이터((3, 10))를 성공적으로 불러왔습니다.


C:\Users\Playdata\AppData\Local\Temp\ipykernel_6592\1965728530.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users = pd.read_sql(users_query, conn)  # ['id', 'caf', 'blend', 'notes', 'sour', 'sweet', 'bitter', 'body', 'user_id']
C:\Users\Playdata\AppData\Local\Temp\ipykernel_6592\1965728530.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings = pd.read_sql(review_query, conn)   # 'id', 'Stars', 'content', 'created_date', 'Coffee_id', 'Order_id', 'user_id'


In [231]:
ratings['userid'] = ratings.merge(users, on='user_id')['index']
ratings['coffeeid'] = ratings.merge(coffees, how='left', left_on='Coffee_id', right_on='id')['coffeeid']

In [232]:
ratings.drop(columns=['user_id', 'Coffee_id'], inplace=True)
coffees.drop(columns='id', inplace=True)
users.drop(columns=['user_id', 'id'], inplace=True)
users.rename(columns={"index":"userid"}, inplace=True)

In [233]:
import json
jsonDec = json.decoder.JSONDecoder()

In [234]:
users

,userid,caf,blend,notes,sour,sweet,bitter,body
0,0,1,1,"[""\ucd08\ucf5c\ub9bf"", ""\uace0\uc18c\ud568""]",2,4,3,3
1,1,1,1,"[""\ucd08\ucf5c\ub9bf"", ""\uace0\uc18c\ud568""]",1,4,3,4
2,2,1,1,"[""\uaf43"", ""\ud5c8\ube0c""]",5,2,4,3


In [235]:
ratings

,Stars,created_date,userid,coffeeid
0,4,2023-11-23 03:55:00.373289,0,0


In [236]:
coffees.head(2)

,coffeeid,body,sour,sweet,bitter,caf,blend,single,지속가능성_0,지속가능성_공정무역,...,로스팅 포인트_라이트미디엄,로스팅 포인트_미디엄,로스팅 포인트_미디엄다크,꽃,과일,허브,달콤함,고소함,향료,초콜릿
0,0,2,3,5,1,1,1,0,0,1,...,0,1,0,0,1,0,0,0,0,1
1,1,4,2,4,1,1,1,0,0,1,...,0,0,1,0,1,0,0,1,0,1


In [237]:
#@title Solution
DOT = 'dot'
COSINE = 'cosine'
def compute_scores(query_embedding, item_embeddings, measure=DOT):
  """Computes the scores of the candidates given a query.
  Args:
    query_embedding: a vector of shape [k], representing the query embedding.
    item_embeddings: a matrix of shape [N, k], such that row i is the embedding
      of item i.
    measure: a string specifying the similarity measure to be used. Can be
      either DOT or COSINE.
  Returns:
    scores: a vector of shape [N], such that scores[i] is the score of item i.
  """
  u = query_embedding
  V = item_embeddings
  if measure == COSINE:
    V = V / np.linalg.norm(V, axis=1, keepdims=True)
    u = u / np.linalg.norm(u)
  scores = u.dot(V.T)
  return scores

In [238]:
df = pd.read_csv('mysite/data/features.csv')

In [239]:
# df_copy['cosine_similarity'] = cosine_similarity_matrix[-1]
# result_df = df_copy[df_copy['id'].notna()].sort_values(by='cosine_similarity', ascending=False)[:top_n]
# return result_df['id'].tolist()

def collaborative_rec(model, measure=DOT, exclude_rated=True, k=8, userid=0):
        scores = compute_scores(
            model.embeddings["userid"][userid], model.embeddings["coffeeid"], measure)
        score_key = measure + ' score'
        df_copy = df.copy()
        df_copy[score_key] = list(scores)
        if exclude_rated:
            # remove movies that are already rated
            rated_coffees = ratings[ratings.userid == str(userid)]["coffeeid"].values
            df_copy.reset_index(drop=False, inplace=True)
            df_copy.rename(columns={'index':'coffeeid'}, inplace=True)
            df_copy = df_copy[df_copy.coffeeid.apply(lambda coffeeid: coffeeid not in rated_coffees)]
        
        result_df = df_copy.sort_values([score_key], ascending=False).head(k)
        return result_df['id'].tolist()

In [241]:
from keras.models import load_model
new_model = load_model('mysite/model/test_model.hdf5')

In [242]:
new_model = load_model('mysite/model/test_model.hdf5')
new_model.embeddings = {
    'userid': new_model.get_layer('user_embedding').weights[0].numpy(), # U (46, 30)
    'coffeeid': new_model.get_layer('coffee_embedding').weights[0].numpy() # V (1682, 30)
}

In [250]:
new_model.embeddings["userid"].shape

(46, 15)

In [247]:
import numpy as np
collaborative_rec(new_model, measure=DOT, k=5, userid=3, exclude_rated=True)

[4482, 4333, 3438, 4122, 4790]

In [173]:
collaborative_rec(new_model, userid=2)

[3508, 3151, 1026, 1475, 3740, 3011, 4694, 1573]

In [244]:
users[users['userid'] == 3]

,userid,caf,blend,notes,sour,sweet,bitter,body


In [174]:
test_list = collaborative_rec(new_model, userid=2)

In [175]:
df[df['id'].isin(test_list)]

,id,body,sour,sweet,bitter,caf,blend,single,지속가능성_0,지속가능성_공정무역,...,로스팅 포인트_라이트미디엄,로스팅 포인트_미디엄,로스팅 포인트_미디엄다크,꽃,과일,허브,달콤함,고소함,향료,초콜릿
113,1026,4,2,3,1,1,1,0,1,0,...,0,0,1,0,1,0,0,1,0,0
172,1475,4,3,4,2,1,1,0,1,0,...,0,0,1,0,0,0,0,1,0,1
182,1573,3,3,4,3,0,0,1,1,0,...,1,0,0,0,0,1,1,0,0,1
212,3011,3,2,4,2,1,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
262,3151,4,1,3,1,1,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
334,3508,4,1,4,3,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
381,3740,3,3,4,1,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
507,4694,5,1,5,4,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


***
dataframes 전처리 후
***

In [176]:
from __future__ import print_function

import numpy as np
import pandas as pd
import collections
# from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

# Install spreadsheets and import authentication module.
USER_RATINGS = True

In [177]:
# Utility to split the data into training and test sets.
def split_dataframe(df, holdout_fraction=0.1):
  """Splits a DataFrame into training and test sets.
  Args:
    df: a dataframe.
    holdout_fraction: fraction of dataframe rows to use in the test set.
  Returns:
    train: dataframe for training
    test: dataframe for testing
  """
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]
  return train, test

In [178]:
# Tensorflow2로 MF 구현

import tensorflow as tf
from tensorflow import keras
from keras.models import Model

from keras.layers import Input, Embedding, Flatten, dot

In [179]:
users.shape[0], coffees.shape[0]  # user 3명, 커피 559개

(3, 559)

### building models with actual users

In [182]:
n_latent_factors = 10 # user와 movie embedding의 차원수

user_input = Input(shape=[1], name='user')
coffee_input = Input(shape=[1], name='coffee')

user_embedding = Embedding(input_dim=users.shape[0] # 3
                           , output_dim = n_latent_factors # 10
                           , name='user_embedding'
                           )(user_input)

coffee_embedding = Embedding(input_dim=coffees.shape[0] # 559
                           , output_dim = n_latent_factors # 10
                           , name='coffee_embedding'
                           )(coffee_input)

user_vec = Flatten(name='flatten_users')(user_embedding) # 1차원 배열로 변환
coffee_vec = Flatten(name='flatten_coffees')(coffee_embedding) # 1차원 배열로 변환

product = dot([coffee_vec, user_vec], axes=1) # 평점
model = Model(inputs=[user_input, coffee_input], outputs=product) # user와 movie가 주어졌을 때, 평점을 예측하는 모델

In [183]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 coffee (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 coffee_embedding (Embeddin  (None, 1, 10)                5590      ['coffee[0][0]']              
 g)                                                                                               
                                                                                                  
 user_embedding (Embedding)  (None, 1, 10)                30        ['user[0][0]']          

In [184]:
# compile
model.compile(optimizer='adam', loss='mse')

In [185]:
ratings['userid'] = ratings['userid'].astype(int)
ratings['coffeeid'] = ratings['coffeeid'].astype(int)

train_ratings, test_ratings = split_dataframe(ratings)
train_ratings.shape, test_ratings.shape

((1, 4), (0, 4))

In [187]:
train_ratings.shape[0]

1

In [188]:
history = model.fit(x=[train_ratings['userid'], train_ratings['coffeeid']],
                    y=train_ratings['Stars'], epochs=500,
                    validation_data=([test_ratings['userid'], test_ratings['coffeeid'] ],
                                     test_ratings['Stars']),
                    verbose=1, batch_size=train_ratings.shape[0])

Epoch 1/500



UnimplementedError: Graph execution error:

Detected at node mean_squared_error/Cast defined at (most recent call last):
  File "C:\Users\Playdata\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\Users\Playdata\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "c:\venvs\coffee\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\venvs\coffee\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance

  File "c:\venvs\coffee\lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "c:\venvs\coffee\lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\Playdata\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 600, in run_forever

  File "C:\Users\Playdata\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 1896, in _run_once

  File "C:\Users\Playdata\AppData\Local\Progr

***
find similar user from survey participants
***

In [251]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import json
# import numpy as np

In [273]:
survey_users = pd.read_csv('userinfo.csv')

In [274]:
survey_users['email'].nunique()

45

In [275]:
survey_users.shape

(46, 9)

In [276]:
# Check for duplicates
# survey_users[survey_users.duplicated('email')]

In [277]:
# survey_users[survey_users['email']=='kimhwnj@hanmail.net']

In [278]:
# survey_users.drop(27, inplace=True)
# survey_users.shape

In [279]:
survey_users = survey_users.reset_index(drop=False)  # 0부터 시작하는 index 생성
survey_users = survey_users.rename(columns={"index":"userid"})
survey_users = survey_users.drop(columns='id')

In [259]:
# column_to_convert = survey_users['CupNoteCategories'].tolist()
# json_string = json.dumps(column_to_convert)
# converted_list = json.loads(json_string)
# survey_users['CupNoteCategories'] = json_string

In [280]:
survey_users.head()

,userid,email,Caffeine,CoffeeType,CupNoteCategories,Body,Sourness,Sweetness,Bitterness
0,0,hwhj1214@naver.com,0,1,"['초콜릿', '고소함']",4,1,4,2
1,1,syjflora@gmail.com,1,0,"['초콜릿', '달콤함', '고소함']",3,3,3,3
2,2,aiwithsj@gmail.com,0,0,['고소함'],4,2,3,3
3,3,katherine@gmail.com,0,0,['고소함'],5,1,1,4
4,4,seunghyun@test.com,0,1,"['달콤함', '고소함']",3,1,5,2


In [281]:
survey_users.drop(columns=['email', 'userid'], inplace=True) 
survey_users = survey_users[['Caffeine', 'CoffeeType', 'CupNoteCategories', 'Sourness', 'Sweetness', 'Bitterness', 'Body']]
survey_users.rename(columns={'Caffeine':'caf', 'CoffeeType':'blend', 'CupNoteCategories':'notes', 'Sourness':'sour', 'Sweetness':'sweet', 'Bitterness':'bitter', 'Body':'body'}, inplace=True)
survey_users.head(2)

,caf,blend,notes,sour,sweet,bitter,body
0,0,1,"['초콜릿', '고소함']",1,4,2,4
1,1,0,"['초콜릿', '달콤함', '고소함']",3,3,3,3


In [282]:
# survey_users export
survey_users.to_csv('mysite/data/survey_users.csv', index=0)
test=pd.read_csv('mysite/data/survey_users.csv')
test.head(2)

,caf,blend,notes,sour,sweet,bitter,body
0,0,1,"['초콜릿', '고소함']",1,4,2,4
1,1,0,"['초콜릿', '달콤함', '고소함']",3,3,3,3


In [283]:
from sqlalchemy import create_engine

# MySQL 연결 정보
mysql_host = 'localhost'
mysql_user = 'root'
mysql_password = 'MShw1214!'
mysql_db = 'wondoodoo'

# Create SQLAlchemy engine
engine = create_engine(f"mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_db}")

review_query = f"SELECT * FROM review;"
users_query = f"SELECT * FROM main_preference;"

try:
    # Read data using SQLAlchemy engine
    users_data = pd.read_sql(users_query, engine)
    ratings_data = pd.read_sql(review_query, engine)

    # Further data processing...
    
except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # Close SQLAlchemy engine
    engine.dispose()


In [284]:
users_data

,id,caf,blend,notes,sour,sweet,bitter,body,user_id
0,1,1,1,"[""\ucd08\ucf5c\ub9bf"", ""\uace0\uc18c\ud568""]",2,4,3,3,1
1,2,1,1,"[""\ucd08\ucf5c\ub9bf"", ""\uace0\uc18c\ud568""]",1,4,3,4,2
2,3,1,1,"[""\uaf43"", ""\ud5c8\ube0c""]",5,2,4,3,3


In [285]:
users_data.reset_index(drop=False, inplace=True)
jsonDec = json.decoder.JSONDecoder()
users_data['notes']=users_data['notes'].apply(jsonDec.decode)

users_data.drop(columns=['user_id', 'id'], inplace=True)
users_data.rename(columns={"index": "userid"}, inplace=True)
users_data.head()

,userid,caf,blend,notes,sour,sweet,bitter,body
0,0,1,1,"[초콜릿, 고소함]",2,4,3,3
1,1,1,1,"[초콜릿, 고소함]",1,4,3,4
2,2,1,1,"[꽃, 허브]",5,2,4,3


In [286]:
users_data.drop(columns='userid', inplace=True)
users_data.head(2)

,caf,blend,notes,sour,sweet,bitter,body
0,1,1,"[초콜릿, 고소함]",2,4,3,3
1,1,1,"[초콜릿, 고소함]",1,4,3,4


In [267]:
survey_users_test = survey_users.copy()

In [268]:
import ast
survey_users_test['notes'] = survey_users_test['notes'].apply(ast.literal_eval)
survey_users_test.head(2)

KeyError: 'notes'

In [269]:
users_data_test = users_data.copy()

In [287]:
def notes_categorising(data):
    try:
        data['notes'] = data['notes'].apply(ast.literal_eval)
    except:
        pass
    # Add other keys with initial values
    new_keys = ['꽃', '과일', '달콤함', '허브', '고소함', '향료', '초콜릿']
    for key in new_keys:
        data[key] = 0
    # Iterate through the 'notes' list and add new keys to 'new_dict'
    for index, row in data.iterrows():
        for note in row['notes']:
            data.at[index, note] = 1
    try:
        del data['notes']
    except:
        pass

In [271]:
notes_categorising(users_data_test)
users_data_test.head(2)

,caf,blend,sour,sweet,bitter,body,꽃,과일,달콤함,허브,고소함,향료,초콜릿
0,1,1,2,4,3,3,0,0,0,0,1,0,1
1,1,1,1,4,3,4,0,0,0,0,1,0,1


In [288]:
userid = 2 # def similar_user(userid):
new_user = users_data[users_data.index==userid]
notes_categorising(new_user)
print(new_user)

df_copy = survey_users.copy()
notes_categorising(df_copy)

df_copy = pd.concat([df_copy, new_user], ignore_index=True)

embeddings = df_copy.values
cosine_similarity_matrix = cosine_similarity(embeddings, embeddings)

df_copy['cosine_similarity'] = cosine_similarity_matrix[-1]
result_df = df_copy[df_copy.index != df_copy.index[-1]].sort_values(by='cosine_similarity', ascending=False)[:1]

# result_df = [df_copy['id'].notna()].sort_values(by='cosine_similarity', ascending=False)[:top_n]

  caf blend sour sweet bitter body  꽃  과일  달콤함  허브  고소함  향료  초콜릿
2   1     1    5     2      4    3  1   0    0   1    0   0    0


C:\Users\Playdata\AppData\Local\Temp\ipykernel_6592\4091565707.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[key] = 0
C:\Users\Playdata\AppData\Local\Temp\ipykernel_6592\4091565707.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[key] = 0
C:\Users\Playdata\AppData\Local\Temp\ipykernel_6592\4091565707.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [289]:
result_df.index[0]

22

In [290]:
users_data[users_data.index==2]

,caf,blend,notes,sour,sweet,bitter,body
2,1,1,"[꽃, 허브]",5,2,4,3
